# 0. 라이브러리로드

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# 1. Queue 자료형 구현

In [ ]:
#Queue의 기본적인 기능 구현
class Queue():
    def __init__(self, maxsize):
        self.queue = []
        self.maxsize = maxsize
        
    # Queue에 Data 넣음
    def enqueue(self, data):
        self.queue.append(data)

    # Queue에 가장 먼저 들어온 Data 내보냄
    def dequeue(self):
        dequeue_object = None
        if self.isEmpty():
            print("Queue is Empty")
        else:
            dequeue_object = self.queue[0]
            self.queue = self.queue[1:]
        return dequeue_object
    
    # Queue에 가장 먼저들어온 Data return
    def peek(self):
        peek_object = None
        if self.isEmpty():
            print("Queue is Empty")
        else:
            peek_object = self.queue[0]
        return peek_object
    
    # Queue가 비어있는지 확인
    def isEmpty(self):
        is_empty = False
        if len(self.queue) == 0:
            is_empty = True
        return is_empty
    
    # Queue의 Size가 Max Size를 초과하는지 확인
    def isMaxSizeOver(self):
        queue_size = len(self.queue)
        if (queue_size > self.maxsize):
            return False
        else :
            return True

# 2. 데이터로드

In [ ]:
# 맞는 경로로 수정해 ! 나중에 합쳐야해서 꼭 인덱스 버전으로 가져와줘
product_df = pd.read_csv('Data/크림_브랜드별_신발_제품_인덱스ver.csv')
product_df.head()

,index,검색어,브랜드,제품명,제품한글명,가격,거래량,관심수,url
0,0,A.P.C.,Nike,Nike x A.P.C. Dunk High '08 SP Beige,나이키 x 아페쎄 덩크 하이 '08 SP 베이지,"280,000원",1,57,https://kream.co.kr/products/34916
1,1,A.P.C.,Nike,Nike x A.P.C. Blazer Mid Vintage White Midnigh...,나이키 x 아페쎄 블레이저 미드 빈티지 화이트 미드나잇 네이비,-,NaN,17,https://kream.co.kr/products/3841
2,2,Ader Error,Puma,Puma x Ader Error Vaderon White,NaN,"260,000원",530,"2,352",https://kream.co.kr/products/54312
3,3,Ader Error,Birkenstock,Birkenstock x Ader Error A630 Black,NaN,"220,000원",311,"1,400",https://kream.co.kr/products/62262
4,4,Ader Error,Camper,(W) Camper x Ader Error Proto-111-S Cinder Bas...,NaN,"410,000원",21,134,https://kream.co.kr/products/36842


# 3. 브랜드 데이터 생성

In [ ]:
# 'Jordan' 자기가 맡은 브랜드로 바꿔서 돌리면 돼 ! 
index_list = product_df[product_df['브랜드']=='Jordan'].index
df = product_df.loc[index_list,:]
df = df.reset_index(drop=True)
df.head()

,index,검색어,브랜드,제품명,제품한글명,가격,거래량,관심수,url
0,804,Camper,Jordan,Jordan 1 Retro High OG Black Mocha,NaN,"570,000원",NaN,2.6만,https://kream.co.kr/products/25922
1,807,Camper,Jordan,Jordan 1 x Travis Scott x Fragment Retro Low O...,NaN,"1,588,000원",NaN,2.7만,https://kream.co.kr/products/38367
2,808,Camper,Jordan,Jordan 1 Low Golf Wolf Grey,NaN,"243,000원",NaN,3.2만,https://kream.co.kr/products/48705
3,810,Camper,Jordan,Jordan 1 Mid Light Smoke Grey Anthracite,NaN,"198,000원",NaN,5.2만,https://kream.co.kr/products/47257
4,818,Camper,Jordan,Jordan 1 Retro High OG Bleached Coral,NaN,"216,000원",NaN,2.6만,https://kream.co.kr/products/60302


# 4. 거래량 데이터 프레임 생성

In [ ]:
transaction_df = pd.DataFrame(columns=['index','제품한글명','제품정보리스트','거래리스트'])
transaction_df

,index,제품한글명,제품정보리스트,거래리스트


# 5. 데이터 크롤링

In [ ]:
# 아무 페이지 들어가서 로그인 해두는거야
driver = webdriver.Chrome()
driver.get('https://kream.co.kr/products/38367') 
time.sleep(3)

In [ ]:
# 로그인
driver.find_element(By.XPATH,'//*[@id="__layout"]/div/div[1]/div[1]/div/ul/li[4]/a').click()
time.sleep(2)
input_list = driver.find_elements(By.CLASS_NAME,'input_txt')
input_list[0].send_keys('') # 여기에 아이디 넣고
input_list[1].send_keys('') # 여기에 비밀번호 넣으면 돼 
time.sleep(1)
driver.find_element(By.CLASS_NAME,'login_btn_box').click()
time.sleep(2)

In [ ]:
index = len(transaction_df) # 거래량 데이터 저장 인덱스 

for idx in tqdm(range(0,len(df),1)):
    url = df.loc[idx,'url']
    driver.get(url)
    time.sleep(1)

    transaction_df.loc[index,'index'] = df.loc[idx,'index']
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "sub_title")))
    transaction_df.loc[index,'제품한글명'] = driver.find_element(By.CLASS_NAME,'sub_title').text

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "detail_product")))
    transaction_df.loc[index,'제품정보리스트'] = driver.find_element(By.CLASS_NAME,'detail_product').text

    # 로그인이 풀린 경우 
    login_buttons = driver.find_elements(By.XPATH,'//*[@id="__layout"]/div/div[2]/div[1]/div/div[2]/div/div[5]/div[1]/div[4]/div/a')
    if len(login_buttons)!=0:
        login_buttons[0].click()
        time.sleep(1)
        wait = WebDriverWait(driver,20)
        input_field = wait.until(EC.presence_of_element_located((By.CLASS_NAME,'input_txt')))
        input_list = driver.find_elements(By.CLASS_NAME,'input_txt')
        input_list[0].send_keys('id') 
        input_list[1].send_keys('pwd') 
        time.sleep(1)
        driver.find_element(By.CLASS_NAME,'login_btn_box').click()
        time.sleep(2)

    # <체결 내역 더보기> 클릭
    time.sleep(1)
    buttons = driver.find_elements(By.XPATH,'//*[@id="panel1"]/a')

    if len(buttons)==0: # 버튼이 아예 없는 경우 
        index = index + 1
        pass
    else:
        wait = WebDriverWait(driver, 20)
        button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="panel1"]/a')))
        driver.find_element(By.XPATH,'//*[@id="panel1"]/a').click()

        # 팝업창 스크롤 내리기
        wait = WebDriverWait(driver,20)
        pop_up = wait.until(EC.visibility_of_element_located((By.CLASS_NAME,'layer_container')))
        price_pop_up = driver.find_element(By.CLASS_NAME,'price_body')
        # 스크롤 끝까지 내리기
        last_height = driver.execute_script("return arguments[0].scrollHeight",price_pop_up)
        szQ = Queue(50)
        enqueue_count  = 0
        while True:
            # 스크롤 내리기
            driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', price_pop_up)
            # scroll height를 가져오는 주기
            time.sleep(2)
            new_page_height = driver.execute_script("return arguments[0].scrollHeight",price_pop_up)

            # queue가 꽉 차는 경우 스크롤 내리기 종료
            if (enqueue_count>szQ.maxsize):
                break
            
            # 첫 Loop 수행 (Queue가 비어있는 경우) 예외 처리
            if(szQ.isEmpty()):
                szQ.enqueue(new_page_height)
                enqueue_count += 1

            # Queue에 가장 먼저 들어온 데이터와 새로 업데이트 된 Scroll Height를 비교함
            # 같으면 그대로 Enqueue, 다르면 Queue의 모든 Data를 Dequeue 후 새로운 Scroll Height를 Enqueue 함.    
            else :
                if(szQ.peek() == new_page_height) :
                    szQ.enqueue(new_page_height)
                    enqueue_count += 1
                else :
                    szQ.enqueue(new_page_height)
                    for z in range(enqueue_count) :
                        szQ.dequeue()
                    enqueue_count = 1
        print('스크롤 내리기 완료')
        # 거래 데이터 가져오기
        # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "body_list")))
        time.sleep(1)
        transactions = driver.find_elements(By.CLASS_NAME,'body_list')
        for i in tqdm(range(len(transactions))):
            transaction_df.loc[index,'index'] = df.loc[idx,'index']
            # transaction_df.loc[index,'제품한글명'] = product_name.text
            # transaction_df.loc[index,'제품정보리스트'] = product_info[0].text
            transaction_df.loc[index,'거래리스트'] = transactions[i].text
            index = index+1     

  0%|          | 0/3057 [00:00<?, ?it/s]

스크롤 내리기 완료


  0%|          | 1/3057 [03:43<189:42:57, 223.49s/it]

스크롤 내리기 완료


  0%|          | 2/3057 [06:46<169:40:25, 199.94s/it]

...

  1%|          | 23/3057 [46:04<78:23:02, 93.01s/it]

스크롤 내리기 완료


  1%|          | 24/3057 [47:57<83:21:04, 98.93s/it]

스크롤 내리기 완료


  1%|          | 25/3057 [49:49<86:44:36, 102.99s/it]

In [ ]:
# 저장할 경로및 이름으로 수정
transaction_df.to_csv('Data/크림_조던_거래량_데이터.csv',index=False,encoding='utf-8-sig')

In [ ]:
# transaction_df.to_csv('Data/조던_테스트_제발마지막.csv',index=False,encoding='utf-8-sig')

In [ ]:
transaction_df.to_csv('Data/조던_테스트_0728.csv',index=False,encoding='utf-8-sig')